In [37]:
import json
import rdflib
import re
import pandas as pd
from rdflib import URIRef, BNode, Literal, Graph, RDF, Namespace, ODRL2, RDFS
import requests
import time
from tqdm import tqdm
import urllib.parse
from maayanlab_bioinformatics.utils import fetch_save_read, merge
from functools import lru_cache
import glob

In [179]:
@lru_cache()
def ncbi_genes_fetch(organism='Mammalia/Homo_sapiens', filters=lambda ncbi: (ncbi['type_of_gene'] == "protein-coding")):
    ''' Fetch the current NCBI Human Gene Info database.
    See ftp://ftp.ncbi.nih.gov/gene/DATA/GENE_INFO/ for the directory/file of the organism of interest.
    '''
    def maybe_split(record):
        ''' NCBI Stores Nulls as '-' and lists '|' delimited
        '''
        if record in {'', '-'}:
            return set()
        return set(record.split('|'))
    #
    def supplement_dbXref_prefix_omitted(ids):
        ''' NCBI Stores external IDS with Foreign:ID while most datasets just use the ID
        '''
        for id in ids:
          # add original id
          yield id
          # also add id *without* prefix
          if ':' in id:
            yield id.split(':', maxsplit=1)[1]
    #
    ncbi = fetch_save_read(
    'ftp://ftp.ncbi.nih.gov/gene/DATA/GENE_INFO/{}.gene_info.gz'.format(organism),
    '{}.gene_info.tsv'.format(organism),
    sep='\t',
    )
    if filters and callable(filters):
        ncbi = ncbi[filters(ncbi)]
    #
    ncbi['All_synonyms'] = [
        set.union(
          maybe_split(gene_info['Symbol']),
          maybe_split(gene_info['Symbol'].upper()),
          maybe_split(gene_info['Symbol_from_nomenclature_authority']),
          maybe_split(str(gene_info['GeneID'])),
          maybe_split(gene_info['Synonyms']),
          maybe_split(gene_info['Other_designations']),
          maybe_split(gene_info['LocusTag']),
          set(supplement_dbXref_prefix_omitted(maybe_split(gene_info['dbXrefs']))),
        )
        for _, gene_info in ncbi.iterrows()
    ]
    return ncbi

@lru_cache()                  
def ncbi_genes_lookup(organism='Mammalia/Homo_sapiens', filters=lambda ncbi: ncbi['type_of_gene']=='protein-coding'):
    ''' Return a lookup dictionary with synonyms as the keys, and official symbols as the values
    Usage:
    ```python
    ncbi_lookup = ncbi_genes_lookup('Mammalia/Homo_sapiens')
    print(ncbi_lookup('STAT3')) # any alias will get converted into the official symbol
    ```
    '''
    ncbi_genes = ncbi_genes_fetch(organism=organism)
    synonyms, symbols, gene_ids = zip(*{
    (synonym, gene_info['Symbol'], gene_info['GeneID'])
    for _, gene_info in ncbi_genes.iterrows()
    for synonym in gene_info['All_synonyms']
    })
    ncbi_lookup = pd.Series(symbols, index=synonyms)
    index_values = ncbi_lookup.index.value_counts()
    ncbi_lookup_disambiguated = ncbi_lookup.drop(index_values[index_values > 1].index)
    for i in symbols:
        if i not in ncbi_lookup_disambiguated.index:
            ncbi_lookup_disambiguated[i] = i
    
    ncbi_lookup_id = pd.Series(gene_ids, index=synonyms)
    index_values_id = ncbi_lookup_id.index.value_counts()
    ncbi_lookup_disambiguated_id = ncbi_lookup_id.drop(index_values_id[index_values_id > 1].index)
    ncbi_genes = ncbi_genes.set_index("GeneID")
    for i in gene_ids:
        gene = ncbi_genes.at[i, "Symbol"]
        if gene not in ncbi_lookup_disambiguated_id.index:
            ncbi_lookup_disambiguated_id[gene] = i

    return ncbi_lookup_disambiguated.to_dict().get, ncbi_lookup_disambiguated_id.get

symbol_mapper, gene_id_mapper = ncbi_genes_lookup()
def get_info(gene):
    return(symbol_mapper(gene), str(gene_id_mapper(gene)))

get_info('NOS1'), get_info('HAUS1P1')

(('NOS1', '4842'), (None, 'None'))

In [121]:
label_id_mapper = {
    "LY-294002": "CID:3973",
    "PD-98059": "CID:4713"
}

In [122]:
gene_scores = pd.read_csv("https://raw.githubusercontent.com/nih-cfde/ReproToxTables/main/Susceptibility%20Scores%20and%20GWAS%20Gene%20Lists/Susceptibility%20Scores/All%20Scores.csv", index_col=0)
gene_scores.head()

,pLI,Residual.Variation.Intolerance.Score,Residual.Variation.Intolerance.Score.Percentile,pHI,pTS
Gene Symbol,,,,,
A1BG,9.064924e-05,-0.466531,23.513800,0.13450,0.52075
A1CF,3.619701e-03,-0.378346,28.013682,0.21197,0.22586
A2M,5.401149e-04,0.099179,60.757254,0.92010,0.39196
A2ML1,1.329022e-22,2.419901,98.525596,0.27137,0.18193
A4GALT,1.115987e-01,-0.290161,33.339231,0.14511,0.09411


In [123]:
drug_scores = pd.read_csv("data/L1000_2021_Signature_Similarity_predicted_drug_table.csv", index_col=0)
drug_scores["Rank"] = drug_scores.Similarity_Score.rank(method="first")
drug_scores.head()

,Similarity_Score,Rank
FTI-276,0.111335,33473.0
BRD-K08703257,0.104131,33472.0
temozolomide,0.102109,33471.0
Gossypetin,0.101613,33470.0
TAK-715,0.100779,33469.0


In [124]:
HPO_Freq = pd.read_csv("data/HPO_Freq.tsv", sep="\t", index_col=1)
HPO_Freq.head()

,HPO description,KF Frequency
HPO ID,,
HP:0410030,Cleft lip,888
HP:0000776,Congenital diaphragmatic hernia,770
HP:0000175,Cleft palate,704
HP:0010438,Abnormal ventricular septum morphology,629
HP:0001710,Conotruncal defect,545


In [125]:
associations = {
  'https://raw.githubusercontent.com/nih-cfde/ReproToxTables/main/CDC-birth-defects/Geneshot_BirthDefects_Gene_Associations.ttl': "Geneshot",
  'https://raw.githubusercontent.com/nih-cfde/ReproToxTables/main/CDC-birth-defects/Drugshot_BirthDefects_Drug_Associations.ttl': "Drugshot",
  'https://raw.githubusercontent.com/nih-cfde/ReproToxTables/main/CDC-birth-defects/DrugEnrichr_BirthDefects_Drug_Associations.ttl': "DrugEnrichr",
}


In [126]:
with open("data/birth_defects.json") as o:
    birth_defects = json.loads(o.read())

with open("data/drug_ref.json") as o:
    drugs = json.loads(o.read())
    for k,v in drugs.items():
        new_dict = {}
        for i,j in v.items():
            new_dict[i.replace(".", "_")] = j
        drugs[k] = new_dict

with open("data/gene_ref.json") as o:
    genes = json.loads(o.read())
    for k,v in genes.items():
        new_dict = {}
        for i,j in v.items():
            new_dict[i.replace(".", "_")] = j
        genes[k] = new_dict

In [127]:
for k,v in genes.items():
    a = v.pop("Residual.Variation.Intolerance.Score", None)
    if a:
        v["Residual_Variation_Intolerance_Score"] = a
    a = v.pop("Residual.Variation.Intolerance.Score.Percentile", None)
    if a:
        v["Residual_Variation_Intolerance_Score_Percentile"] = a

In [128]:
iri_mappers = [
  dict(
    cls='BirthDefect',
    expr=re.compile(r'^https://www\.orpha\.net/ORDO/(?P<id>.+)$'),
  ),
  dict(
    cls='BirthDefect',
    expr=re.compile(r'^https://purl\.obolibrary\.org/obo/(?P<id>.+)$'),
  ),
  dict(
    cls='Gene',
    expr=re.compile(r'^https://identifiers\.org/hgnc\.symbol/(?P<symbol>.+)$'),
  ),
  dict(
    cls='Drug',
    expr=re.compile(r'^https://identifiers\.org/lincs\.smallmolecule:(?P<id>.+)$'),
  ),
  # dict(
  #   cls='Relationship',
  #   expr=re.compile(r'^https://semanticscience\.org/resource/(?P<id>.+)$'),
  # ),
  dict(
    cls='BirthDefect',
    expr=re.compile(r'^https://hpo\.jax\.org/app/browse/term/(?P<id>.+)$'),
  ),
]

relationships = {
  "SIO_010299": "disease",
  "SIO_010056": "phenotype",
  "SIO_000993": "chemical-disease",
  "SIO_000983": "gene-disease"
}

orpha = {}
def iri_to_node(iri):
    for mapper in iri_mappers:
        iri = iri.replace("HP_", "HP:")
        m = mapper['expr'].match(iri)
        if m:
            group_dict = m.groupdict()
            if mapper['cls'] == "Gene":
              properties = {
                "id": genes[group_dict["symbol"]]["id"],
                "label": genes[group_dict["symbol"]]["label"]
              }
              if group_dict["symbol"] in gene_scores:
                print(group_dict["symbol"])
                for k,v in gene_scores.loc[group_dict["symbol"]]:
                  properties[k.replace(".", "_" )] = v
              gene_symbol, gene_id = get_info(group_dict["symbol"])
              if gene_symbol and gene_id:
                properties["label"] = gene_symbol
                properties["id"] = gene_id
                properties["uri"] = "https://www.ncbi.nlm.nih.gov/gene/%s"%gene_id
                
                
              else:
                properties["uri"] = "https://uswest.ensembl.org/Homo_sapiens/Gene/Summary?g=%s"%group_dict["symbol"]
              return { "type": mapper['cls'], "properties": {**properties}}
            elif mapper['cls'] == "Drug":
              gd = drugs[group_dict["id"]]
              if not str(drugs[group_dict["id"]]["id"]).startswith("LSM"):
                gd["id"] = str(drugs[group_dict["id"]]["id"])
                gd["uri"] = "https://pubchem.ncbi.nlm.nih.gov/compound/%s"%str(group_dict["id"])
              return {"type": mapper['cls'], "properties": gd}
            elif mapper['cls'] == "BirthDefect":
              birth_defect_id = group_dict["id"]
              if birth_defect_id in birth_defects:
                birth_defect = birth_defects[birth_defect_id]
                properties = {
                  "id": birth_defect_id,
                  "label": birth_defect,
                  "uri": "https://purl.obolibrary.org/obo/=%s"%birth_defect
                }
                return { "type": mapper['cls'], "properties": {**properties}}
              elif birth_defect_id in orpha:
                properties = orpha[birth_defect_id]
                return { "type": mapper['cls'], "properties": {**properties}}
              else:
                res = requests.get("https://www.ebi.ac.uk/ols/api/select?q=%s"%birth_defect_id)
                if not res.ok:
                  print("ERROR %s"%birth_defect_id)
                  print(res.text)
                else:
                  print(birth_defect_id)
                  birth_defect = res.json()["response"]["docs"][0]["label"]
                  birth_defect_id = res.json()["response"]["docs"][0]["obo_id"]
                  properties = {
                    "id": birth_defect_id,
                    "label": birth_defect,
                    "uri": res.json()["response"]["docs"][0]["iri"]
                  }
                  orpha[birth_defect_id.replace(":", "_")] = properties
                  return { "type": mapper['cls'], "properties": {**properties}}
            else:
              print(group_dict)

              
              
            # if "id" in group_dict:
            #     if group_dict["id"] in birth_defects:
            #         group_dict["label"] = birth_defects[group_dict["id"]]
            #         return {"type": mapper['cls'], "properties": group_dict}
            #     elif group_dict["id"] in drugs:
            #         gd = drugs[group_dict["id"]]
            #         if mapper['cls'] == 'Drug':
            #           gd["id"] = "CID:%s"%str(drugs[group_dict["id"]]["id"])
            #         return {"type": mapper['cls'], "properties": gd}
            #     elif group_dict["id"] in relationships:
            #         group_dict["label"] = relationships[group_dict["id"]]
            #         return {"type": mapper['cls'], "properties": group_dict}
            # elif "symbol" in group_dict:
            #     gene_symbol, gene_id = get_info(group_dict["symbol"])
            #     if gene_symbol and gene_id:
            #       properties = genes[group_dict["symbol"]]
            #       properties["label"] = gene_symbol
            #       properties["id"] = gene_id
            #       properties["uri"] = "https://www.ncbi.nlm.nih.gov/gene/%s"%gene_id
            #       return { "type": mapper['cls'], "properties": {**group_dict, **properties}}
            #     else:
            #       return { "type": mapper['cls'], "properties": {**group_dict, 
            #         **genes[group_dict["symbol"]],
            #         "uri": "https://uswest.ensembl.org/Homo_sapiens/Gene/Summary?g=%s"%group_dict["symbol"]
            #       }}
            # return {"type": mapper['cls'], "properties": group_dict}


In [129]:
edges = []
nodes = {}
hpo_ids = set()
no_freq = set()
new_ids = {}
for assoc, resource in associations.items():
    rdfgraph = rdflib.Graph()
    rdfgraph.parse(assoc, format='ttl')
    for i in rdfgraph.all_nodes():
        iri = iri_to_node(i)
        if iri and str(iri["properties"]["id"]) not in nodes:
            if iri["type"] == "Drug":
                # iri["properties"]["uri"] = "https://pubchem.ncbi.nlm.nih.gov/compound/%s"%str(iri["properties"]["id"].replace("CID:", ""))
                uid = str(iri["properties"]["id"])
                label = iri["properties"]["label"]
                if label in label_id_mapper:
                    if uid not in new_ids:
                        new_ids[uid] = label_id_mapper[label]
                    uid = label_id_mapper[label]
                else:
                    label_id_mapper[label] = uid
                    new_ids[uid] = uid
                iri["properties"]["id"] = uid
                nodes[uid] = {
                    **iri,
                }
            elif iri["type"] == "Gene":
                uid = str(iri["properties"]["id"])
                label = iri["properties"]["label"]
                if label in label_id_mapper:
                    if uid not in new_ids:
                        new_ids[uid] = label_id_mapper[label]
                    uid = label_id_mapper[label]
                else:
                    label_id_mapper[label] = uid
                    new_ids[node_id] = uid
                iri["properties"]["id"] = uid
                nodes[uid] = {
                    **iri,
                    
                }
            elif iri["type"] == "BirthDefect" and str(iri["properties"]["id"].startswith("HP:")):
                # iri["properties"]["uri"] = "https://purl.obolibrary.org/obo/=%s"%str(iri["properties"]["id"])
                node_id = str(iri["properties"]["id"])
                label = iri["properties"]["label"]
                if label in label_id_mapper:
                    if node_id not in new_ids:
                        new_ids[node_id] = label_id_mapper[label]
                    node_id = label_id_mapper[label]
                else:
                    label_id_mapper[label] = node_id
                    new_ids[node_id] = node_id
                iri["properties"]["id"] = node_id
                nodes[node_id] = {
                    **iri,
                }
                if node_id in HPO_Freq.index:
                    nodes[node_id]["KF_freq"] = int(HPO_Freq.at[str(iri["properties"]["id"]), 'KF Frequency'])
                    hpo_ids.add(node_id)
                else:
                    no_freq.add(node_id)
            else:
                uid = str(iri["properties"]["id"])
                label = iri["properties"]["label"]
                if label in label_id_mapper:
                    if uid not in new_ids:
                        new_ids[uid] = label_id_mapper[label]
                    uid = label_id_mapper[label]
                else:
                    label_id_mapper[label] = uid
                    new_ids[uid] = uid
                iri["properties"]["id"] = uid
                nodes[str(iri["properties"]["id"])] = {
                    **iri,
                    "uri": str(i)
                }
    for subj, pred, obj in rdfgraph.triples((None, None, None)):
        subj_iri = iri_to_node(subj)
        obj_iri = iri_to_node(obj)
        if subj_iri and nodes[str(subj_iri["properties"]["id"])] and obj_iri and nodes[str(obj_iri["properties"]["id"])]:
            pred = str(pred)
            n = iri_to_node(pred)
            if n and n["id"]:
                pred = relationships[n["id"]]
            
            edges.append({
                "type": "Relation",
                "source": new_ids[str(subj_iri["properties"]["id"])],
                "relation": "%s"%resource,#(resource, relationships[pred.split("/")[-1]]),
                "target": new_ids[str(obj_iri["properties"]["id"])],
                "properties": {
                    "id": "%s_%s_%s"%(subj_iri["properties"]["label"], relationships[pred.split("/")[-1]], obj_iri["properties"]["label"]),
                    "source_label": subj_iri["properties"]["label"],
                    "target_label": obj_iri["properties"]["label"],
                    "directed": True,
                    "resource": resource
                }
            })


TypeError: 'dict' object is not callable

In [ ]:
serialization_v1 = {
    "version": "1",
    "nodes": nodes,
    "edges": edges
}
with open("results/reprotox_serialization.v1.json", "w") as o:
    o.write(json.dumps(serialization_v1, indent=2))

In [ ]:
def isNumber(value):
    try:
        v = int(value)
        return {"@type": "int", "@value": v}
    except:
        try:
            v = float(value)
            return {"@type": "number", "@value": v}
        except:
            return False


In [ ]:
isinstance('', str)

True

In [130]:
def typer(value):
    numb = isNumber(str(value))
    if numb:
        return numb
    elif isinstance(value, str):
        return {
            "@type": "string",
            "@value": value
        }
    elif isinstance(value, list):
        type_list = []
        for i in value:
            type_list.append(typer(i))
        
        return {
            "@type": "array",
            "@value": type_list
        }
    elif isinstance(value, dict):
        type_dict = {}
        for k,v in value.items():
            if k == "id" or k == "target" or k == "source":
                type_dict[k] = {
                    "@type": "string",
                    "@value": v
                }
            else: type_dict[k] = typer(v)
        return {
            "@type": "object",
            "@value": type_dict
        }
    

In [131]:
nodes_v2 = {}
for k,v in nodes.items():
    nodes_v2[k] = typer(v)

In [132]:
edges_v2 = []
for i in edges:
    edges_v2.append(
        typer(i)
    )

In [133]:
serialization_v2 = {
    "version": "2",
    "nodes": nodes_v2,
    "edges": edges_v2
}
with open("results/reprotox_serialization.v2.json", "w") as o:
    o.write(json.dumps(serialization_v2, indent=2))

In [134]:
def get_id(value):
    for prefix, val in rdfgraph.namespaces():
        v = value.replace(val, "")
        if not v == value:
            return prefix, v
    else:
        return "Literal", value


## SigCom LINCS
### drug2gene

In [227]:
with open("data/SigComLINCS_drug_2_gene_25.v1.json") as o:
    sigcom_lincs = json.loads(o.read())

In [228]:
gene_names = set()
for k,v in sigcom_lincs["nodes"].items():
    t = v["type"]
    if t == "Gene":
        gene_names.add(v["properties"]["label"])
len(gene_names)

4420

In [229]:
payload = {
    "filter": {
        "where": {
            "meta.symbol": {"inq": list(gene_names)}
        }
    }
}
res = requests.post("https://maayanlab.cloud/sigcom-lincs/metadata-api/entities/find", json=payload)
len(res.json())

4411

In [230]:
gene_mapping_name = {
    'LINC00341': 'SYNE3',
    'ATP5MPL': 'ATP5MJ'
}
gene_mapping = {
    'SYNE3': '161176',
    'ATP5MJ': '9556'
}
for i in res.json():
    gene_id = i["meta"]["geneid"]
    gene_symbol = i["meta"]["symbol"]
    gene_mapping[gene_symbol] = gene_id

In [231]:
for i in gene_names - gene_mapping.keys():
    gene_symbol, gene_id = get_info(i)
    gene_mapping[gene_symbol] = str(gene_id)

In [232]:
nodes = {}
node_map = {}
ensembl_map = {}
id_mapper = {}
for k,v in sigcom_lincs["nodes"].items():
    t = v["type"]
    v["properties"]["id"] = str(v["properties"]["id"].replace("CID", "CID:"))
    new_v = {}
    for i,j in v["properties"].items():
        new_v[i.replace(".", "_")] = j
    label = new_v["label"]
    if t == "Gene":
        old_id = v["properties"]["id"]
        gene_label = gene_mapping_name.get(label, label)
        gene_id = gene_mapping.get(gene_label)
        if gene_label and gene_id:
            uid = gene_id
            if label in label_id_mapper:
                uid = label_id_mapper[label]
            else:
                label_id_mapper[label] = uid
            id_mapper[gene_id] = uid
            ensembl_map[old_id] = uid
            new_v["id"] = uid
            new_v["label"] = gene_label
            new_v["uri"] = "https://www.ncbi.nlm.nih.gov/gene/%s"%new_v["id"]
            label = gene_label
            if label in gene_scores.index:
                for key,val in gene_scores.loc[label].items():
                    new_v[key.replace(".", "_")] = val
        else:
            print(label)
            continue
    elif t == "Drug":
        uid = new_v["id"]
        if label in label_id_mapper:
            uid = label_id_mapper[label]
        else:
            label_id_mapper[label] = uid
        id_mapper[new_v["id"]] = uid
        new_v["id"] = uid
        new_v["uri"] = "https://pubchem.ncbi.nlm.nih.gov/compound/%s"%v["properties"]["id"].replace("CID:", "")
        if label in drug_scores.index:
            new_v["placenta_score"] = drug_scores.at[label, "Similarity_Score"]
            new_v["placenta_rank"] = drug_scores.at[label, "Rank"]
    nodes[new_v["id"]] = {
        "type": t,
        "properties": new_v
    }
    node_map[k] = str(v["properties"]["id"])
    node_map[str(new_v["id"])] = str(v["properties"]["id"])

FAM30A


In [233]:
edges = []
not_in = set()
in_set = set()
for i in sigcom_lincs["edges"]:
    source = id_mapper[str(i["source"].replace("CID", "CID:"))]
    if str(i["target"]) in ensembl_map:
        target = id_mapper[ensembl_map[str(i["target"])]]
        # if source not in node_map: not_in.add(source)
        # else: in_set.add(source)
        # if target not in node_map: not_in.add(target)
        # else: in_set.add(target)
        i["source"] = source
        i["target"] = target
        source_label = nodes[source]["properties"]["label"]
        target_label = nodes[target]["properties"]["label"]
        i["properties"] = {
            "id": "%s %s %s"%(source_label, i["relation"], target_label),
            "source_label": source_label,
            "target_label": target_label,
            "resource": "SigCom LINCS",
            "mean_CD_coefficient": i.pop("weight"),
            "directed": True,
        }
        i["relation"] = "SigCom LINCS Drug-to-Gene (%s)"%i["relation"]
        edges.append(i)

In [234]:
serialization_v1 = {
    "version": "1",
    "nodes": nodes,
    "edges": edges
}
with open("results/sigcom_lincs_serialization.v1.json", "w") as o:
    o.write(json.dumps(serialization_v1, indent=2))

In [235]:
nodes_v2 = {}
for k,v in nodes.items():
    nodes_v2[k] = typer(v)

edges_v2 = []
for i in edges:
    edges_v2.append(
        typer(i)
    )
    
serialization_v2 = {
    "version": "2",
    "nodes": nodes_v2,
    "edges": edges_v2
}
with open("results/sigcom_lincs_serialization.v2.json", "w") as o:
    o.write(json.dumps(serialization_v2, indent=2))

### Gene 2 Drug (Should I do this?)

In [ ]:
with open("data/full_weights_SigComLINCS_gene_2_drug07262022.v1.json") as o:
    sigcom_lincs = json.loads(o.read())

In [ ]:
nodes = {}
node_map = {}
for k,v in sigcom_lincs["nodes"].items():
    t = v["type"]
    v["properties"]["id"] = str(v["properties"]["id"].replace("CID", "CID:"))
    new_v = {}
    for i,j in v["properties"].items():
        new_v[i.replace(".", "_")] = j
    label = new_v["label"]
    if t == "Gene":
        new_v["id"] = v["properties"]["id"]
        new_v["label"] = v["properties"]["label"]
        new_v["uri"] = "https://www.ncbi.nlm.nih.gov/gene/%s"%new_v["id"]
        if label in gene_scores.index:
            for key,val in gene_scores.loc[label].items():
                new_v[key.replace(".", "_")] = val
    elif t == "Drug":
        new_v["uri"] = "https://pubchem.ncbi.nlm.nih.gov/compound/%s"%v["properties"]["id"].replace("CID:", "")
        if label in drug_scores.index:
            new_v["placenta_score"] = drug_scores.at[label, "Similarity_Score"]
            new_v["placenta_rank"] = drug_scores.at[label, "Rank"]
    nodes[new_v["id"]] = {
        "type": t,
        "properties": new_v
    }
    node_map[k] = str(v["properties"]["id"])
    node_map[str(new_v["id"])] = str(v["properties"]["id"])

In [ ]:
edges = []
not_in = set()
in_set = set()
for i in sigcom_lincs["edges"]:
    source = str(i["source"].replace("CID", "CID:"))
    target = str(i["target"])
    # if source not in node_map: not_in.add(source)
    # else: in_set.add(source)
    # if target not in node_map: not_in.add(target)
    # else: in_set.add(target)
    i["source"] = source
    i["target"] = target
    source_label = nodes[source]["properties"]["label"]
    target_label = nodes[target]["properties"]["label"]
    i["properties"] = {
        "id": "%s %s %s"%(source_label, i["relation"], target_label),
        "source_label": source_label,
        "target_label": target_label,
        "resource": "SigCom LINCS",
        "mean_CD_coefficient": i.pop("weight"),
        "directed": True,
    }
    i["relation"] = "SigCom LINCS (%s)"%i["relation"]
    edges.append(i)

In [ ]:
edges[0]

In [ ]:
payload = {
    "filter": {
        "where": {
            "meta.symbol": {
                "inq": list(not_in)
            }
        }
    }
}

res = requests.post("https://maayanlab.cloud/sigcom-lincs/metadata-api/entities/find", json=payload)
res.ok

In [ ]:
len(not_in), len(res.json())

In [ ]:
not_in - set([i["meta"]["symbol"] for i in res.json()])

In [ ]:
n = set()
for i in res.json():
    if "ensemblid" not in i["meta"]:
        gene_id = str(i["meta"]["geneid"])
        label = i["meta"]["symbol"]
        nodes[gene_id] = {
            "type": "Gene",
            "properties": {
                "id": gene_id,
                "label": label
            }
        }
    else:
        gene_id = str(i["meta"]["ensemblid"])
        label = i["meta"]["symbol"]
        nodes[gene_id] = {
            "type": "Gene",
            "properties": {
                "id": gene_id,
                "label": label
            }
        }
    node_map[label] = gene_id
    node_map[gene_id] = gene_id

In [ ]:
sigcom_lincs["edges"][0]

In [ ]:
edges = []
not_in = {}
in_set = set()
for i in sigcom_lincs["edges"]:
    source = str(i["source"])
    target = str(i["target"])
    
    if source in nodes and target in nodes:
        i["source"] = node_map.get(source, source)
        i["target"] = node_map.get(target, target)
        cd = i["properties"].pop("Mean CD-coefficient", None)
        if cd:
            i["properties"]["mean_CD_coefficient"] = cd
            source_label = nodes[i["source"]]["properties"]["label"]
            target_label = nodes[i["target"]]["properties"]["label"]
            i["properties"]["source_label"] = source_label
            i["properties"]["target_label"] = target_label
            i["properties"]["id"] = "%s_%s_%s"%(source_label, i["relation"], target_label)
            i["relation"] = "SigCom LINCS (%s)"%i["relation"]
        edges.append(i)
    elif not source in nodes:
        if source not in not_in:
            not_in[source] = 0
        not_in[source]+=1
    elif not target in nodes:
        if target not in not_in:
            not_in[target] = 0
        not_in[target]+=1

In [ ]:
edges[1]

In [ ]:
len(edges), len(sigcom_lincs["edges"])

In [ ]:
serialization_v1 = {
    "version": "1",
    "nodes": nodes,
    "edges": edges
}
with open("results/sigcom_lincs_serialization.v1.json", "w") as o:
    o.write(json.dumps(serialization_v1, indent=2))

In [ ]:
nodes_v2 = {}
for k,v in nodes.items():
    nodes_v2[k] = typer(v)


In [ ]:
edges_v2 = []
for i in edges:
    edges_v2.append(
        typer(i)
    )

In [ ]:
nodes_v2 = {}
for k,v in nodes.items():
    nodes_v2[k] = typer(v)

edges_v2 = []
for i in edges:
    edges_v2.append(
        typer(i)
    )
    
serialization_v2 = {
    "version": "2",
    "nodes": nodes_v2,
    "edges": edges_v2
}
with open("results/sigcom_lincs_serialization.v2.json", "w") as o:
    o.write(json.dumps(serialization_v2, indent=2))

### Drug2Drug

In [142]:
with open("data/full_SigComLINCS_lm_drug_cosine_sim.v1.json") as o:
    drug2drug = json.loads(o.read())

In [143]:
nodes = {}
id_mapper = {}
for k,v in drug2drug["nodes"].items():
    properties = v["properties"]
    properties["id"] = k.replace("CID", "CID:")
    if k == "CID4549":
        print(properties["id"])
    label = properties["label"]
    uid = properties["id"]
    if label in label_id_mapper:
        id_mapper[uid] = label_id_mapper[label]
        uid = label_id_mapper[label]
    else:
        id_mapper[uid] = uid
        label_id_mapper[label] = uid
    properties["id"] = uid

    id_mapper[properties["id"]] = uid
    properties["uri"] = "https://pubchem.ncbi.nlm.nih.gov/compound/%s"%v["properties"]["id"].replace("CID:", "")
    if label in drug_scores.index:
        properties["placenta_score"] = drug_scores.at[label, "Similarity_Score"]
        properties["placenta_rank"] = drug_scores.at[label, "Rank"]
    nodes[properties["id"]] = {
        "type": v["type"],
        "properties": properties
    }
    

CID:4549


In [144]:
edges = []
for i in drug2drug["edges"]:
    source = id_mapper[i["source"].replace("CID", "CID:")]
    source_label = nodes[source]["properties"]["label"]
    target = id_mapper[i["target"].replace("CID", "CID:")]
    target_label = nodes[target]["properties"]["label"]
    relation = "LINCS Drugs Cosine Similarity"
    cosine_similarity = i["weight"]

    edges.append({
        "source": source,
        "relation": relation,
        "target": target,
        "properties": {
            "id": "%s-%s similarity"%(source_label, target_label),
            "source_label": source_label,
            "target_label": target_label,
            "cosine_similarity": cosine_similarity
        }
    })


In [145]:
drug2drug["nodes"]['CID4549']

{'type': 'Drug',
 'properties': {'id': 'CID:4549',
  'label': 'NPPB',
  'uri': 'https://pubchem.ncbi.nlm.nih.gov/compound/4549',
  'placenta_score': 0.016815556,
  'placenta_rank': 21011.0}}

In [147]:
serialization_v1 = {
    "version": "1",
    "nodes": nodes,
    "edges": edges
}
with open("results/sigcom_lincs_drug_similarity.v1.json", "w") as o:
    o.write(json.dumps(serialization_v1, indent=2))

In [148]:
nodes_v2 = {}
for k,v in nodes.items():
    nodes_v2[k] = typer(v)

edges_v2 = []
for i in edges:
    edges_v2.append(
        typer(i)
    )

serialization_v2 = {
    "version": "2",
    "nodes": nodes_v2,
    "edges": edges_v2
}
with open("results/sigcom_lincs_drug_similarity.v2.json", "w") as o:
    o.write(json.dumps(serialization_v2, indent=2))

## Drug Target

In [149]:
drug_target = pd.read_csv("data/idg_target.tsv", sep="\t")
drug_target.head()

,DRUG_SMILES,DRUG_NAME,DRUG_ID,DRUG_PUBCHEM_CID,TARGET_NAME,TARGET_CLASS,TARGET_GENE,TARGET_ACCESSION,TARGET_SWISSPROT,TARGET_ORGANISM,TARGET_TDL,ACT_TYPE,ACT_RELATION,ACT_VALUE,ACT_COMMENT,ACT_SOURCE
0,CCN(CC)CCNC(=O)C1=C(C)NC(\C=C2/C(=O)NC3=C2C=C(...,sunitinib,2544,5329102,AP2-associated protein kinase 1,Kinase,AAK1,Q2M2I8,AAK1_HUMAN,Homo sapiens,Tchem,Kd,=,7.96,Binding constant for AAK1 kinase domain,CHEMBL
1,CN(C)C[C@@H]1CCN2C=C(C3=CC=CC=C23)C2=C(C(=O)NC...,ruboxistaurin,3533,153999,AP2-associated protein kinase 1,Kinase,AAK1,Q2M2I8,AAK1_HUMAN,Homo sapiens,Tchem,Kd,=,6.05,Binding constant for AAK1 kinase domain,CHEMBL
2,N#CC[C@H](C1CCCC1)N1C=C(C=N1)C1=C2C=CNC2=NC=N1,ruxolitinib,4190,25126798,AP2-associated protein kinase 1,Kinase,AAK1,Q2M2I8,AAK1_HUMAN,Homo sapiens,Tchem,Kd,=,6.92,Binding constant for AAK1 kinase domain,CHEMBL
3,COC(=O)C1=CC2=C(C=C1)\C(=C(\NC1=CC=C(C=C1)N(C)...,nintedanib,4903,135423438,AP2-associated protein kinase 1,Kinase,AAK1,Q2M2I8,AAK1_HUMAN,Homo sapiens,Tchem,Kd,=,7.20,Binding constant for AAK1 kinase domain,CHEMBL
4,CCS(=O)(=O)N1CC(CC#N)(C1)N1C=C(C=N1)C1=NC=NC2=...,baricitinib,5202,44205240,AP2-associated protein kinase 1,Kinase,AAK1,Q2M2I8,AAK1_HUMAN,Homo sapiens,Tchem,Kd,=,7.77,Binding affinity determined in a cell-free bio...,IUPHAR


In [150]:
nodes = {}
edges = []

In [157]:
for drug_id in set(drug_target["DRUG_ID"]):
    sub_df = drug_target[drug_target["DRUG_ID"] == drug_id]
    for i in sub_df.index:
        drug_pubchem_id = "CID:%s"%sub_df.at[i, "DRUG_PUBCHEM_CID"]
        drug_name = sub_df.at[i, "DRUG_NAME"]
        if drug_name in label_id_mapper:
            drug_pubchem_id = label_id_mapper[drug_name]
        else:
            label_id_mapper[drug_name] = drug_pubchem_id
        if drug_pubchem_id not in nodes:
            properties = {
                "id": drug_pubchem_id,
                "label": drug_name,
                "smiles": sub_df.at[i, "DRUG_SMILES"],
                "IDG_ID": str(sub_df.at[i, "DRUG_ID"]),
                "uri": "https://pubchem.ncbi.nlm.nih.gov/compound/%s"%str(sub_df.at[i, "DRUG_PUBCHEM_CID"])
            }
            if drug_name in drug_scores.index:
                properties["placenta_score"] = float(drug_scores.at[drug_name, "Similarity_Score"])
                properties["placenta_rank"] = int(drug_scores.at[drug_name, "Rank"])
            nodes[drug_pubchem_id] = {
                "type": "Drug",
                "properties": properties
            }
        
        gene_label = sub_df.at[i, "TARGET_GENE"]
        gene_label, gene_id = get_info(gene_label)
        if gene_label and gene_id:
            if gene_id not in nodes:
                properties = {
                    "id": gene_id,
                    "label": gene_label,
                    "target_name": str(sub_df.at[i, "TARGET_NAME"]),
                    "target_class": str(sub_df.at[i, "TARGET_CLASS"]),
                    "accession": str(sub_df.at[i, "TARGET_ACCESSION"]),
                    "swissprot": str(sub_df.at[i, "TARGET_SWISSPROT"]),
                    "organism": str(sub_df.at[i, "TARGET_ORGANISM"]),
                    "TDL": str(sub_df.at[i, "TARGET_TDL"])
                }
                if gene_label in gene_scores.index:
                    for k,v in gene_scores.loc[gene_label].items():
                        properties[k.replace(".", "_")] = float(v)
                nodes[gene_id] = {
                    "type": "Gene",
                    "properties": properties
                }
            edge = {
                "source": drug_pubchem_id,
                "relation": "IDG (Drug Target)",
                "target": gene_id,
                "properties": {
                    "id": "%s targets %s"%(drug_name, gene_label),
                    "source_label": drug_name,
                    "target_label": gene_label,
                    "act_type": str(sub_df.at[i, "ACT_TYPE"]),
                    "act_relation": str(sub_df.at[i, "ACT_RELATION"]),
                    "act_value": str(sub_df.at[i, "ACT_VALUE"]),
                    "act_comment": str(sub_df.at[i, "ACT_COMMENT"]),
                    "act_source": str(sub_df.at[i, "ACT_SOURCE"]),
                }
            }
            edges.append(edge)
        else:
            print(sub_df.at[i, "TARGET_GENE"])

MT-CO2
MT-CO2


In [158]:
drug_target.shape

(7326, 16)

In [159]:
serialization_v1 = {
    "version": "1",
    "nodes": nodes,
    "edges": edges
}
with open("results/idg_drug_targets.v1.json", "w") as o:
    o.write(json.dumps(serialization_v1, indent=2))

In [160]:
nodes_v2 = {}
for k,v in nodes.items():
    nodes_v2[k] = typer(v)

edges_v2 = []
for i in edges:
    edges_v2.append(
        typer(i)
    )

serialization_v2 = {
    "version": "2",
    "nodes": nodes_v2,
    "edges": edges_v2
}
with open("results/idg_drug_targets.v2.json", "w") as o:
    o.write(json.dumps(serialization_v2, indent=2))

### HPO

In [ ]:
HPO_Freq.head()

In [ ]:
nodes = {}
edges = []
for i in HPO_Freq.index:
    if i not in nodes:
        label = HPO_Freq.at[i, "HPO description"]
        if type(label) == pd.core.series.Series:
            label = label[0]
        KF_Freq =  HPO_Freq.at[i, "KF Frequency"]
        if type(KF_Freq) == pd.core.series.Series:
            KF_Freq = int(KF_Freq[0])
        else:
            KF_Freq = int(KF_Freq)
        nodes[i] = {
            "type": "BirthDefect",
            "properties": {
                "id": i,
                "label": label,
                "KF_Freq": KF_Freq,
                "uri": "https://purl.obolibrary.org/obo/%s"%i.replace(":", "_")
            }
        }
        time.sleep(0.1)
        res = requests.get("https://hpo.jax.org/api/hpo/term/%s/genes?max=-1&offset=1"%i, verify=False)
        if not res.ok:
            print(i)
        else:
            for gene_info in res.json()["genes"]:
                entrezGeneId = str(gene_info["entrezGeneId"])
                entrezGeneSymbol = gene_info["entrezGeneSymbol"]
                if entrezGeneId not in nodes:
                    properties = {
                        "id": entrezGeneId,
                        "label": entrezGeneSymbol,
                        "uri": "https://www.ncbi.nlm.nih.gov/gene/%s"%entrezGeneId
                    }
                    if entrezGeneSymbol in gene_scores:
                        for k,v in gene_scores.loc[entrezGeneSymbol]:
                            properties[k.replace(".", "_" )] = v
                    
                    nodes[entrezGeneId] = {
                        "type": "Gene",
                        "properties": properties
                    }
                edge = {
                    "source": i,
                    "relation": "HPO",
                    "target": entrezGeneId,
                    "properties": {
                        "id": "%s (%s-%s)"%(i, label, entrezGeneSymbol),
                        "source_label": label,
                        "target_label": entrezGeneSymbol,
                        "resource": "HPO"
                    }
                }
                edges.append(edge)

In [ ]:
serialization_v1 = {
    "version": "1",
    "nodes": nodes,
    "edges": edges
}
with open("results/hpo.v1.json", "w") as o:
    o.write(json.dumps(serialization_v1, indent=2))

In [ ]:
nodes_v2 = {}
for k,v in nodes.items():
    nodes_v2[k] = typer(v)

edges_v2 = []
for i in edges:
    edges_v2.append(
        typer(i)
    )

serialization_v2 = {
    "version": "2",
    "nodes": nodes_v2,
    "edges": edges_v2
}
with open("results/hpo.v2.json", "w") as o:
    o.write(json.dumps(serialization_v2, indent=2))

In [161]:
for i in glob.glob("results/*.v1.json"):
    with open(i) as o:
        with open(i.replace("results", "ingestion").replace("v1", "valid"), 'w') as w:
            w.write(o.read())

In [164]:
duplicates = {}
for i in glob.glob("results/*.v1.json"):
    with open(i) as o:
        serialization = json.loads(o.read())
        for k,v in serialization["nodes"].items():
            uid = v["properties"]["id"]
            label = v["properties"]["label"]
            if label in duplicates and not duplicates[label] == uid:
                print(label, uid, duplicates[label])
            else:
                duplicates[label] = uid


NPPB CID:4549 4879


### ARCHS4

In [180]:
import pyarrow.feather as feather
from scipy.stats import rankdata, zscore

In [181]:
human_correlation = feather.read_feather('data/human_correlation_archs4.f')

In [182]:
human_correlation.index = human_correlation.columns
human_correlation.head()

,A1BG,A1CF,A2M,A2ML1,A2MP1,A4GALT,A4GNT,AAAS,AACS,AACSP1,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
A1BG,1.000000,0.311017,0.074197,0.011767,0.015465,-0.083539,0.024190,-0.023043,0.116815,0.005853,...,-0.020775,-0.065653,-0.034081,-0.002724,-0.022020,0.101489,0.020808,-0.093609,-0.025960,0.003271
A1CF,0.311017,1.000000,0.314577,-0.001876,-0.000682,-0.065426,0.020387,-0.047918,0.046401,-0.004944,...,-0.079724,-0.071680,-0.018917,0.047886,0.003180,0.002353,-0.007614,-0.086449,0.018525,0.003854
A2M,0.074197,0.314577,1.000000,-0.028479,-0.017056,0.014158,0.014082,-0.021786,-0.059868,-0.022951,...,-0.097068,-0.067448,0.001960,0.018967,0.046186,-0.111606,-0.013873,0.028855,0.021196,-0.037100
A2ML1,0.011767,-0.001876,-0.028479,1.000000,0.007315,0.038877,-0.005643,-0.024810,0.058989,0.031632,...,-0.021674,-0.047888,0.005581,0.009300,-0.008702,0.031462,0.013790,-0.055190,0.001249,0.018487
A2MP1,0.015465,-0.000682,-0.017056,0.007315,1.000000,-0.035422,-0.008135,-0.002369,-0.003118,0.011400,...,-0.039716,-0.023186,0.007086,-0.010465,0.026512,0.061912,0.003802,-0.027152,0.010331,0.035132


In [183]:
human_correlation_rank = pd.DataFrame(0, columns=human_correlation.columns, index=human_correlation.columns)

In [184]:
import numpy as np

mat = np.zeros(shape=(len(human_correlation.columns), len(human_correlation.columns)))

In [186]:
index = 0
for col in tqdm(human_correlation.columns):
    mat[index] = rankdata(human_correlation[col], method="ordinal")
    index += 1

human_correlation_rank = pd.DataFrame(mat.T, columns=human_correlation.columns, index=human_correlation.columns)

100%|██████████| 26415/26415 [01:24<00:00, 311.73it/s]


In [187]:
human_correlation_rank.head()

,A1BG,A1CF,A2M,A2ML1,A2MP1,A4GALT,A4GNT,AAAS,AACS,AACSP1,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
A1BG,26415.0,26139.0,24341.0,19615.0,17890.0,1504.0,24806.0,7732.0,24667.0,15673.0,...,8400.0,3239.0,3643.0,15276.0,8709.0,23701.0,18611.0,4081.0,3907.0,14736.0
A1CF,26213.0,26415.0,26290.0,11596.0,12135.0,3123.0,24412.0,2522.0,21136.0,10264.0,...,1906.0,2680.0,8014.0,24413.0,17256.0,10458.0,12104.0,4600.0,21128.0,14911.0
A2M,23681.0,26148.0,26415.0,2019.0,4307.0,19063.0,23302.0,8125.0,1845.0,3078.0,...,1186.0,3072.0,16801.0,21126.0,23215.0,757.0,10281.0,20651.0,21551.0,3505.0
A2ML1,14082.0,15952.0,8867.0,26415.0,15361.0,21329.0,9210.0,7215.0,22181.0,22530.0,...,8213.0,5514.0,17860.0,19058.0,13515.0,16049.0,17304.0,7903.0,16350.0,18637.0
A2MP1,15198.0,16392.0,12952.0,17723.0,26415.0,7577.0,6113.0,15240.0,10237.0,17718.0,...,5174.0,10441.0,18232.0,11824.0,21397.0,20649.0,15113.0,13100.0,19301.0,21531.0


In [188]:
organism='Mammalia/Homo_sapiens'
ncbi = fetch_save_read(
    'ftp://ftp.ncbi.nih.gov/gene/DATA/GENE_INFO/{}.gene_info.gz'.format(organism),
    '{}.gene_info.tsv'.format(organism),
    sep='\t',
)
ncbi = ncbi.set_index("GeneID")

In [189]:
ncbi.head()

,#tax_id,Symbol,LocusTag,Synonyms,dbXrefs,chromosome,map_location,description,type_of_gene,Symbol_from_nomenclature_authority,Full_name_from_nomenclature_authority,Nomenclature_status,Other_designations,Modification_date,Feature_type
GeneID,,,,,,,,,,,,,,,
1,9606,A1BG,-,A1B|ABG|GAB|HYST2477,MIM:138670|HGNC:HGNC:5|Ensembl:ENSG00000121410...,19,19q13.43,alpha-1-B glycoprotein,protein-coding,A1BG,alpha-1-B glycoprotein,O,alpha-1B-glycoprotein|HEL-S-163pA|epididymis s...,20220703,-
2,9606,A2M,-,A2MD|CPAMD5|FWP007|S863-7,MIM:103950|HGNC:HGNC:7|Ensembl:ENSG00000175899...,12,12p13.31,alpha-2-macroglobulin,protein-coding,A2M,alpha-2-macroglobulin,O,alpha-2-macroglobulin|C3 and PZP-like alpha-2-...,20220703,-
3,9606,A2MP1,-,A2MP,HGNC:HGNC:8|Ensembl:ENSG00000256069|AllianceGe...,12,12p13.31,alpha-2-macroglobulin pseudogene 1,pseudo,A2MP1,alpha-2-macroglobulin pseudogene 1,O,pregnancy-zone protein pseudogene,20220513,-
9,9606,NAT1,-,AAC1|MNAT|NAT-1|NATI,MIM:108345|HGNC:HGNC:7645|Ensembl:ENSG00000171...,8,8p22,N-acetyltransferase 1,protein-coding,NAT1,N-acetyltransferase 1,O,arylamine N-acetyltransferase 1|N-acetyltransf...,20220522,-
10,9606,NAT2,-,AAC2|NAT-2|PNAT,MIM:612182|HGNC:HGNC:7646|Ensembl:ENSG00000156...,8,8p22,N-acetyltransferase 2,protein-coding,NAT2,N-acetyltransferase 2,O,arylamine N-acetyltransferase 2|N-acetyltransf...,20220703,-


In [190]:
nodes = {} 
edges = {}
no_data = set()
gene_types = {}
for gene in tqdm(human_correlation_rank.index):
    # check if we want this gene
    gene_symbol, gene_id = get_info(gene)
    if gene_symbol and gene_id:
        if gene_id not in nodes:
            synonyms = list(set(ncbi.at[int(gene_id), "Synonyms"].split("|")))
            XRefs = {}
            gene_type = ncbi.at[int(gene_id), "type_of_gene"]
            if gene_type not in gene_types:
                gene_types[gene_type] = 0
            gene_types[gene_type] += 1
            for i in ncbi.at[int(gene_id), "dbXrefs"].split("|"):
                split = i.split(":")
                k = split[0]
                v = ":".join(split[1:])
                XRefs[k] = v
            scores = {}
            if gene_symbol in gene_scores.index:
                for k,v in gene_scores.loc[gene_symbol].items():
                    scores[k.replace(".", "_")] = v
            
            nodes[gene_id] = {
                "type": "Gene",
                "properties": {
                    "id": gene_id,
                    "label": gene_symbol,
                    "uri": "https://www.ncbi.nlm.nih.gov/gene/%s"%gene_id,
                    **scores
                }
            }
#             if len(synonyms):
#                 nodes[gene_id]["properties"]["synonyms"] = synonyms
#             if len(XRefs):
#                 nodes[gene_id]["properties"]["XRefs"] = ["%s: %s"%(k,v) for k,v in XRefs.items()]
        rank = 0
        count = 0
        sorted_genes = human_correlation_rank[gene].sort_values().index
        while count < 5:
            coexpressed_gene = sorted_genes[rank]
            coexpressed_symbol, coexpressed_id = get_info(coexpressed_gene)
            if coexpressed_symbol and coexpressed_id:
                if coexpressed_id not in nodes:
                    gene_type = ncbi.at[int(coexpressed_id), "type_of_gene"]
                    if gene_type not in gene_types:
                        gene_types[gene_type] = 0
                    gene_types[gene_type] += 1
                    synonyms = list(set(ncbi.at[int(coexpressed_id), "Synonyms"].split("|")))
                    XRefs = {}
                    for i in ncbi.at[int(coexpressed_id), "dbXrefs"].split("|"):
                        split = i.split(":")
                        k = split[0]
                        v = ":".join(split[1:])
                        XRefs[k] = v
                    scores = {}
                    if coexpressed_symbol in gene_scores.index:
                        for k,v in gene_scores.loc[coexpressed_symbol].items():
                            scores[k.replace(".", "_")] = v
                    nodes[coexpressed_id] = {
                        "type": "Gene",
                        "properties": {
                            "id": coexpressed_id,
                            "label": coexpressed_symbol,
                            "uri": "https://www.ncbi.nlm.nih.gov/gene/%s"%coexpressed_id,
                            **scores
                        }
                    }
#                     if len(synonyms):
#                         nodes[coexpressed_id]["properties"]["synonyms"] = synonyms
#                     if len(XRefs):
#                         nodes[coexpressed_id]["properties"]["XRefs"] = ["%s: %s"%(k,v) for k,v in XRefs.items()]
                    
                edge_1 = (gene_id, coexpressed_id)
                edge_2 = (coexpressed_id, gene_id)
                if edge_1 not in edges and edge_2 not in edges:
                    edges[edge_1] = {
                        "type": "Relation",
                        "source": gene_id,
                        "relation": "ARCHS4 (negatively correlated)",
                        "target": coexpressed_id,
                        "properties": {
                            "resource": "ARCHS4",
                            "source_label": gene_symbol,
                            "target_label": coexpressed_symbol,
                            "label": "negatively correlated",
                            "id": "%s is negatively correlated with %s"%(gene_symbol, coexpressed_symbol),
                            "correlation_coefficient": human_correlation.at[coexpressed_gene, gene]
                        }
                    }
                count += 1
            rank += 1
            
        count = 0
        rank = len(sorted_genes) - 1
        
        while count < 5:
            coexpressed_gene = sorted_genes[rank]
            if not gene == coexpressed_gene:
                coexpressed_symbol, coexpressed_id = get_info(coexpressed_gene)
                if coexpressed_symbol and coexpressed_id:
                    if coexpressed_id not in nodes:
                        nodes[coexpressed_id] = {
                            "type": "Gene",
                            "properties": {
                                "id": coexpressed_id,
                                "label": coexpressed_symbol
                            }
                        }
                    edge_1 = (gene_id, coexpressed_id)
                    edge_2 = (coexpressed_id, gene_id)
                    if edge_1 not in edges and edge_2 not in edges:
                        edges[edge_1] = {
                            "type": "Relation",
                            "source": gene_id,
                            "relation": "ARCHS4 (positively correlated)",
                            "target": coexpressed_id,
                            "properties": {
                                "resource": "ARCHS4",
                                "source_label": gene_symbol,
                                "target_label": coexpressed_symbol,
                                "label": "positively correlated",
                                "id": "%s is positively correlated with %s"%(gene_symbol, coexpressed_symbol),
                                "correlation_coefficient": human_correlation.at[coexpressed_gene, gene]
                            }
                        }
                    count += 1
            rank -= 1
    else:
        no_data.add(gene)
print(len(no_data))

100%|██████████| 26415/26415 [01:23<00:00, 317.11it/s] 

8422


In [191]:
gene_types

{'protein-coding': 10230}

In [192]:
len(nodes), len(edges)

(17964, 170801)

In [193]:
serialization_v1 = {
    "version": "1",
    "nodes": nodes,
    "edges": list(edges.values())
}
with open("results/archs4_coexpression.v1.json", "w") as o:
    o.write(json.dumps(serialization_v1, indent=2))

nodes_v2 = {}
for k,v in tqdm(nodes.items()):
    nodes_v2[k] = typer(v)
edges_v2 = []
for i in tqdm(edges.values()):
    edges_v2.append(
        typer(i)
    )

serialization_v2 = {
    "version": "2",
    "nodes": nodes_v2,
    "edges": edges_v2
}
with open("results/archs4_coexpression.v2.json", "w") as o:
    o.write(json.dumps(serialization_v2, indent=2))

100%|██████████| 170801/170801 [00:06<00:00, 24436.40it/s]


### copy

In [237]:
for i in glob.glob('results/*.v1.json'):
    with open(i) as o:
        new_file = i.replace("v1", "valid").replace('results', 'ingestion')
        with open(new_file, 'w') as w:
            w.write(o.read())

In [239]:
for i in glob.glob('results/*.v1.json'):
    with open(i) as o:
        serialization = json.loads(o.read())
        print("file: %s, nodes: %s, edges: %s"%(i, len(serialization["nodes"]),len(serialization["edges"])))

file: results/reprotox_serialization.v1.json, nodes: 1427, edges: 2252
file: results/drugsto_faers_female.v1.json, nodes: 126, edges: 193
file: results/sigcom_lincs_drug_similarity.v1.json, nodes: 4523, edges: 20785
file: results/drugsto_faers_male.v1.json, nodes: 126, edges: 193
file: results/hpo.v1.json, nodes: 5195, edges: 126862
file: results/sigcom_lincs_serialization.v1.json, nodes: 8942, edges: 225509
file: results/archs4_coexpression.v1.json, nodes: 17964, edges: 170801
file: results/idg_drug_targets.v1.json, nodes: 2394, edges: 7324
